<h3>Implementation of picture classification using fast ai</h3>

##### The code requires that the images are saved in a folder structure.
 For questions contact: paul.freihaut@psychologie.uni.freiburg.de


In [13]:
# import core modules
from fastai.vision import *
from fastai.metrics import accuracy
import numpy as np
from random import shuffle

##### Helper Function to create splitted datasets for cross validation

In [30]:
# split all participant data into 5 folds

def create_cv_folds(path, namelen):
    
    fnames = get_image_files(path)
    
    print(fnames[0].stem)
    
    participant_ids = []

    for file in fnames:
        if file.stem[namelen:] not in participant_ids:
            participant_ids.append(file.stem[namelen:])


    # randomize particpant ids
    shuffle(participant_ids)

    folds = np.array_split(participant_ids, 5)
    len(participant_ids)
    print(folds[0])
    return folds

#### Use the images of the mouse usage data for condition classification using the fastai library

In [ ]:
# create a Image DataBunch containing the following steps:
# 1. Where are the inputs and how to create them
# 2. How to split the data into training and validation sets
# 3. How to label the inputs
# 4. What transforms to apply
# 5. How to wrap in dataloaders and create the DataBunch

# Classify the mouse usage pictures in the different tasks

# folder refers to the name of the folder in which the images are stored, namelenght to the lenght of the image file names

# Point Click task:
folder, namelen = "Images_PointClick_Task", 14
# Drag Drop task:
# folder, namelen = "Images_DragDrop_Task", 12
# Drawing task:
# folder, namelen = "Images_Drawing_Task", 11
# Follow- Box task:
# folder, namelen = "Images_FollowBox_Task", 13

print(folder + " Classification using FastAI")

path = "your_path/" + folder


# get the splits for the five-fold-cross-validation
folds = create_cv_folds(path, namelen)


# create a regular expression that grabs the correct data classes
pat = r'/([^/]+)_\d+.png$'

# create a 5 fold-group validation loop

for i in range(len(folds)):
    
    print("Using fold " + str(i) + " to validate and other folds to train")
    
    # Create a databunch using the the validation set from the folds split
    data = (ImageList.from_folder(path)
            .split_by_valid_func(lambda o: o.stem[namelen:] in folds[i])
            .label_from_re(pat)
            .transform(size=224) # default 224
            .databunch()
            .normalize()
    )

    # Print the class names
    print(data.classes)

    # print information about the training and validation dataset
    # print(data.dl)

    # use this to show the images that are fed into the learner
    # data.show_batch(rows=3, figsize=(7,6))
    
    # initiate the learner
    learn = cnn_learner(data, models.resnet34, metrics=accuracy)
    
    # train and validate
    learn.fit_one_cycle(3)
    

##### Save the results of the analysis

In [40]:
# results from fastai analysis

point_click = [50.00, 50.00, 44.44, 50.00, 50.00]
print("PointClick: ", np.mean(point_click), np.std(point_click))
drag_drop = [50.00, 60.00, 50.00, 50.00]
print("DragDrop: ", np.mean(drag_drop), np.std(drag_drop))
drawing = [50.00, 42.85, 50.00, 64.29, 50.00]
print("Drawing: ", np.mean(drawing), np.std(drawing))
follow_box = [50.00, 50.00, 50.00, 50.00, 50.00]
print("FollowBox: ", np.mean(follow_box), np.std(follow_box))
loading_task = [54.54, 50.00, 55.00, 60.00, 65.00]
print("LoadingTask:", np.mean(loading_task), np.std(loading_task))

PointClick:  48.888 2.224000000000001
DragDrop:  52.5 4.330127018922194
Drawing:  51.428 7.001866608269543
FollowBox:  50.0 0.0
LoadingTask: 56.907999999999994 5.13827364004682
